### Data Class

In [1]:
import random

class Sentiment:
    NEGATIVE = 'NEGATIVE'
    NEUTRAL = 'NEUTRAL'
    POSITIVE = 'POSTIVE'

class Review:
    def __init__(self, text,score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
        
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else: # Score of 4 & 5
            return Sentiment.POSITIVE

class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews
        
    def get_text(self):
        return [x.text for x in self.reviews]

    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
        
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)

### Load the Data

In [2]:
import json
file_name = 'Books_small_10000.json'

reviews = []
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))

# Reviews is a list of object of  Review class

In [3]:
print(reviews[5].text)
reviews[5].score

I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia's trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character's voice on a strong subject and making it so that other peoples story may be heard through Mia's.


5.0

In [4]:
len(reviews)

10000

## Prep Data

In [5]:
from sklearn.model_selection import train_test_split
training, test = train_test_split(reviews, test_size = 0.33, random_state=42)

train_container = ReviewContainer(training)
test_container = ReviewContainer(test)

In [6]:
train_container.evenly_distribute()
train_x = train_container.get_text()
train_y = train_container.get_sentiment()

test_container.evenly_distribute()
test_x = test_container.get_text()
test_y = test_container.get_sentiment()

train_x[0]

'I found the story interesting and engaging but the writing was highly self conscious.  There were inconsistencies in the story and several times I was left thinking, &#34;yes, and?&#34; The many untied ends and abrupt changes in the narrative make for an unsatisfying read. Love the idea but the execution was mediocre at best.'

In [7]:
print(train_y.count(Sentiment.POSITIVE))
print(train_y.count(Sentiment.NEGATIVE))

436
436


### Bag of Words Vectorization

In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = TfidfVectorizer(binary=True)
train_x_vectors = vectorizer.fit_transform(train_x)
# Vectorizing each training samples

# not fit again because it is our test data
test_x_vectors = vectorizer.transform(test_x)

print(train_x[0])
print(train_x_vectors[0])
# This shows the training data and the vector used to represent it
## To see in a more traditional way go train_x_vectors[0].toarray()

I found the story interesting and engaging but the writing was highly self conscious.  There were inconsistencies in the story and several times I was left thinking, &#34;yes, and?&#34; The many untied ends and abrupt changes in the narrative make for an unsatisfying read. Love the idea but the execution was mediocre at best.
  (0, 862)	0.13899509632055654
  (0, 611)	0.08906744303659031
  (0, 5003)	0.2499154178485469
  (0, 2837)	0.2181841813501538
  (0, 3957)	0.16011443338155837
  (0, 4782)	0.1070798303643513
  (0, 6392)	0.07287512551659982
  (0, 8390)	0.23914174744987066
  (0, 416)	0.0902784004323492
  (0, 3177)	0.06467339820007935
  (0, 4861)	0.13089755835659625
  (0, 5287)	0.2181841813501538
  (0, 1351)	0.20877247629097237
  (0, 153)	0.22395711187042153
  (0, 2664)	0.18078935723925554
  (0, 8402)	0.26510005342799603
  (0, 4891)	0.11935169731606085
  (0, 8870)	0.19222587537202845
  (0, 81)	0.11368810970355564
  (0, 7970)	0.16904290697088156
  (0, 4616)	0.1538582713914324
  (0, 8039)	

In [9]:
train_x_vectors

<872x8906 sparse matrix of type '<class 'numpy.float64'>'
	with 53647 stored elements in Compressed Sparse Row format>

## Classifiers

### Linear SVM

In [10]:
from sklearn import svm

clf_svm = svm.SVC(kernel='linear')
clf_svm.fit(train_x_vectors, train_y)

print(test_x[0])
clf_svm.predict(test_x_vectors[0])

Helen Smith is a joke of an &#34;academic,&#34; a misogynist and a terrible writer. She doesn't use any good data and scientific research to back up her assumptions about how men have it so tough these days. It's mostly just a book of her ranting and using anecdotal stories about men that she then uses to assume all men must feel this way. It's an insult to the intelligence of anyone reading this book. And in fact, she just reinforces existing gender stereotypes such as &#34;women manipulate men with their superior verbal skills.&#34;If you are interested in this topic, there are far better books to read than this one. Also, I have reason to believe that she is manipulating the Amazon reviews to get positive reviews. Amazon, you should ban her!


array(['POSTIVE'], dtype='<U8')

## Decision Tree Classifier

In [11]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)

DecisionTreeClassifier()

## Gaussian Naive Bayes

In [12]:
from sklearn.naive_bayes import GaussianNB

clf_gnb = GaussianNB()
clf_gnb.fit(train_x_vectors.toarray(), train_y)
# using .toarray method to convert into a dense numpy array b/c train_x_vector is a 
## sparse matrix

GaussianNB()

## Logistic Regression

In [13]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()
clf_log.fit(train_x_vectors, train_y)


LogisticRegression()

## Evaluation

In [14]:
# mean Accuracy
print(clf_svm.score(test_x_vectors, test_y))
print(clf_dec.score(test_x_vectors, test_y))
print(clf_gnb.score(test_x_vectors.toarray(), test_y))
print(clf_log.score(test_x_vectors, test_y))

0.8197115384615384
0.6634615384615384
0.6682692307692307
0.8485576923076923


In [15]:
# F1 scores
from sklearn.metrics import f1_score

print('svm',f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE]))
print('Decision Tree',f1_score(test_y, clf_dec.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE]))
print('Gausian Navie Bayes',f1_score(test_y, clf_gnb.predict(test_x_vectors.toarray()), average=None, labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE]))
print('Logistic Regression',f1_score(test_y, clf_log.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE]))

svm [0.82100239 0.81840194]
Decision Tree [0.66346154 0.66346154]
Gausian Navie Bayes [0.65841584 0.67757009]
Logistic Regression [0.84745763 0.849642  ]


In [24]:
test_set = ['very brillinat', 'bad book do not buy', 'horrible waste of time']
new_test = vectorizer.transform(test_set)

clf_svm.predict(new_test)

array(['POSTIVE', 'NEGATIVE', 'NEGATIVE'], dtype='<U8')

## Tuning our model (with Grid Search)


In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [29]:

# Define pipelines for each model
pipelines = {
    'SVM': Pipeline([('svm', SVC())]),
    'Decision Tree': Pipeline([('dt', DecisionTreeClassifier())]),
    'Gaussian Naive Bayes': Pipeline([('gnb', GaussianNB())]),
    'Logistic Regression': Pipeline([('lr', LogisticRegression())])
}

# Define parameter grids for grid search
param_grids = {
    'SVM': {'svm__C': [0.1, 1, 10], 'svm__gamma': [0.1, 0.01, 0.001], 'svm__kernel': ['rbf', 'linear']},
    'Decision Tree': {'dt__max_depth': [None, 5, 10, 15]},
    'Gaussian Naive Bayes': {},
    'Logistic Regression': {'lr__C': [0.1, 1, 10]}
}

# Perform grid search cross-validation for each model
for model_name, pipeline in pipelines.items():
    print(f"Performing Grid Search CV for {model_name}...")
    grid_search = GridSearchCV(pipeline, param_grids[model_name], cv=5, scoring='accuracy',error_score='raise')
    grid_search.fit(train_x_vectors.toarray(), train_y)
    
    # Print the best parameters and best score
    print(f"Best Parameters: {grid_search.best_params_}")
    print(f"Best Accuracy: {grid_search.best_score_:.4f}")
    
    # Evaluate the model on the test set
    test_accuracy = grid_search.score(test_x_vectors.toarray(), test_y)
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print()


Performing Grid Search CV for SVM...
Best Parameters: {'svm__C': 1, 'svm__gamma': 0.1, 'svm__kernel': 'linear'}
Best Accuracy: 0.8521
Test Accuracy: 0.8197

Performing Grid Search CV for Decision Tree...
Best Parameters: {'dt__max_depth': 5}
Best Accuracy: 0.6904
Test Accuracy: 0.6442

Performing Grid Search CV for Gaussian Naive Bayes...
Best Parameters: {}
Best Accuracy: 0.6364
Test Accuracy: 0.6683

Performing Grid Search CV for Logistic Regression...
Best Parameters: {'lr__C': 1}
Best Accuracy: 0.8521
Test Accuracy: 0.8486



## Saving Model

#### Save model

In [31]:
import pickle

with open('sentiment_classifier.pkl', 'wb') as f:
    pickle.dump(clf_svm, f)


#### Save Vectorizer

In [32]:
# Save the vectorizer to a file
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)